In [1]:
import json, numpy as np, pandas as pd, torch, torch.nn as nn, joblib
from pathlib import Path

In [2]:
TIME_COL   = "TIMESTAMP"
TARGET_COL = "TARGETVAR"
BASE_FEATS = ["U10","V10","U100","V100"]
LAGS_Y     = [1,3,6,12,24]
LAGS_SPEED = [1,3,6]
ROLLS_Y    = [6,12,24]
DEVICE     = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def add_engineered_features(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out["speed10"]  = np.sqrt(out["U10"]**2  + out["V10"]**2)
    out["speed100"] = np.sqrt(out["U100"]**2 + out["V100"]**2)
    d10  = np.arctan2(out["V10"],  out["U10"])
    d100 = np.arctan2(out["V100"], out["U100"])
    out["dir10_sin"], out["dir10_cos"]   = np.sin(d10),  np.cos(d10)
    out["dir100_sin"], out["dir100_cos"] = np.sin(d100), np.cos(d100)
    out["shear_speed"] = out["speed100"] - out["speed10"]
    veer = d100 - d10
    out["veer_sin"], out["veer_cos"] = np.sin(veer), np.cos(veer)
    out["hour"] = pd.to_datetime(out[TIME_COL]).dt.hour
    out["day"]  = pd.to_datetime(out[TIME_COL]).dt.dayofyear
    out["hour_sin"] = np.sin(2*np.pi*out["hour"]/24.0)
    out["hour_cos"] = np.cos(2*np.pi*out["hour"]/24.0)
    out["day_sin"]  = np.sin(2*np.pi*out["day"]/366.0)
    out["day_cos"]  = np.cos(2*np.pi*out["day"]/366.0)
    for L in LAGS_Y:
        out[f"y_lag{L}"] = out[TARGET_COL].shift(L)
    for W in ROLLS_Y:
        out[f"y_roll{W}"] = out[TARGET_COL].shift(1).rolling(W, min_periods=W).mean()
    for L in LAGS_SPEED:
        out[f"speed10_lag{L}"]  = out["speed10"].shift(L)
        out[f"speed100_lag{L}"] = out["speed100"].shift(L)
    return out

def build_feat_list():
    return (
        BASE_FEATS +
        ["speed10","speed100","dir10_sin","dir10_cos","dir100_sin","dir100_cos",
         "shear_speed","veer_sin","veer_cos","hour_sin","hour_cos","day_sin","day_cos"] +
        [f"y_lag{L}" for L in LAGS_Y] +
        [f"y_roll{W}" for W in ROLLS_Y] +
        [f"speed10_lag{L}" for L in LAGS_SPEED] +
        [f"speed100_lag{L}" for L in LAGS_SPEED]
    )

In [3]:
class BiLSTMRegressor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout, bidirectional=True):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size, hidden_size=hidden_size, num_layers=num_layers,
            batch_first=True, dropout=dropout if num_layers>1 else 0.0,
            bidirectional=bidirectional
        )
        out_size = hidden_size * (2 if bidirectional else 1)
        self.norm = nn.LayerNorm(out_size)
        self.head = nn.Sequential(nn.Linear(out_size, out_size), nn.GELU(), nn.Dropout(dropout), nn.Linear(out_size,1))
    def forward(self, x):
        o,_ = self.lstm(x); last = self.norm(o[:, -1, :]); return self.head(last)

In [4]:
def load_artifacts(model_root: str):
    root = Path(model_root)
    with open(root / "../biLSTM/best_params.json","r") as f: best_params = json.load(f)
    xsc = joblib.load(root / "../biLSTM/x_scaler_optuna.pkl")
    ysc = joblib.load(root / "../biLSTM/y_scaler_optuna.pkl")
    state = torch.load(root / "../biLSTM/bilstm_optuna_best.pt", map_location=DEVICE)
    return best_params, xsc, ysc, state

In [5]:
def predict_next_from_last24(model_root: str, last24_df: pd.DataFrame, future_weather: dict|None=None):
    """
    last24_df: DataFrame with at least 24 recent rows and columns:
        TIMESTAMP, TARGETVAR, U10, V10, U100, V100 (same units/schema as training).
    future_weather (optional): dict with keys 'U10','V10','U100','V100' for the *next* hour.
        If provided, we’ll use these exogenous values for the t+1 features.
        If None, we 'hold' the last known exogenous values.
    """
    best, xsc, ysc, state = load_artifacts(model_root)

    df = last24_df.copy().sort_values(TIME_COL).reset_index(drop=True)

    # If the model’s lookback > provided rows, we can’t predict
    lookback = int(best["lookback"])
    if len(df) < lookback:
        raise ValueError(f"Need at least {lookback} rows of history; got {len(df)}.")

    # Optionally append a synthetic next-hour exogenous row (same timestamp +1h)
    if future_weather is None:
        fut = df.iloc[[-1]][[TIME_COL]+BASE_FEATS].copy()
        fut[TIME_COL] = pd.to_datetime(fut[TIME_COL]) + pd.Timedelta(hours=1)
    else:
        fut = pd.DataFrame([{
            TIME_COL: pd.to_datetime(df[TIME_COL].iloc[-1]) + pd.Timedelta(hours=1),
            "U10": future_weather["U10"], "V10": future_weather["V10"],
            "U100": future_weather["U100"], "V100": future_weather["V100"],
        }])
    # set TARGETVAR for the future row temporarily with NaN; we’ll fill it using lags after FE
    fut[TARGET_COL] = np.nan

    # Build a small working frame = history + placeholder next hour
    work = pd.concat([df[[TIME_COL, TARGET_COL]+BASE_FEATS], fut], ignore_index=True)

    # For engineered y-lag/roll features, we need actual history TARGETVAR (available in df).
    # After FE, the last row will have all lags computed from history; TARGETVAR itself is NaN for that last row.
    dfe = add_engineered_features(work)

    # Drop only rows that are still incomplete *before* the last row
    dfe_hist = dfe.iloc[:-1].dropna().copy()
    if len(dfe_hist) < lookback:
        raise ValueError(f"After feature lags/rolls, not enough rows to form a {lookback}-step window. "
                         f"Provide a bit more history (≥ {lookback+1} rows).")

    # The final feature row we’ll predict on is the very last row (t+1), which has complete lags from history
    feat_cols = build_feat_list()
    X_hist = dfe_hist[feat_cols].to_numpy(np.float32)

    # scale with training scalers
    X_hist_s = xsc.transform(X_hist)

    # Build the input window (last `lookback` rows)
    X_window = X_hist_s[-lookback:, :]                       # shape (lookback, n_feats)
    xb = torch.from_numpy(X_window[None, ...]).float().to(DEVICE)

    # Rebuild model & load weights
    model = BiLSTMRegressor(
        input_size=X_window.shape[-1],
        hidden_size=int(best["hidden"]),
        num_layers=int(best["layers"]),
        dropout=float(best["dropout"]),
        bidirectional=bool(best["bidir"])
    ).to(DEVICE)
    model.load_state_dict(state)
    model.eval()

    with torch.no_grad():
        yhat_s = model(xb).cpu().numpy()                    # scaled
    # Invert scaling (and log if used)
    yhat = ysc.inverse_transform(yhat_s).ravel()[0]
    if bool(best["log_target"]):
        yhat = np.expm1(yhat)

    next_ts = pd.to_datetime(df[TIME_COL].iloc[-1]) + pd.Timedelta(hours=1)
    return next_ts, float(yhat)

def _rebuild_model(input_size: int, best_params: dict) -> BiLSTMRegressor:
    model = BiLSTMRegressor(
        input_size=input_size,
        hidden_size=int(best_params["hidden"]),
        num_layers=int(best_params["layers"]),
        dropout=float(best_params["dropout"]),
        bidirectional=bool(best_params["bidir"])
    ).to(DEVICE)
    return model

def predict_over_file(model_root: str, data_path: str) -> pd.DataFrame:
    """
    Run prediction over a whole dataset (same schema as training) and return a DataFrame with:
      TIMESTAMP, y_true, y_pred
    Notes:
      - Uses saved x/y scalers and best_params (lookback, log_target, etc.)
      - Aligns outputs after 'lookback' valid rows post-FE
    """
    # load artifacts
    best, xsc, ysc, state = load_artifacts(model_root)
    lookback = int(best["lookback"])
    log_tgt  = bool(best["log_target"])

    # load data
    if data_path.lower().endswith((".xlsx", ".xls")):
        df = pd.read_excel(data_path)
    else:
        df = pd.read_csv(data_path)
    df[TIME_COL] = pd.to_datetime(df[TIME_COL], errors="coerce", infer_datetime_format=True)
    df = df.sort_values(TIME_COL).reset_index(drop=True)

    # FE and target transform (for scaler shape only — true target kept separately)
    dfe = add_engineered_features(df).dropna().reset_index(drop=True)
    feat_cols = build_feat_list()
    X_all = dfe[feat_cols].to_numpy(np.float32)

    # make y_true in original space (no scaler), but also scaled for inference inversion
    y_true_orig = dfe[TARGET_COL].to_numpy(np.float32).reshape(-1, 1)
    y_for_scaler = np.log1p(np.clip(y_true_orig, a_min=0, a_max=None)) if log_tgt else y_true_orig

    # scale features/targets
    Xs = xsc.transform(X_all)
    ys = ysc.transform(y_for_scaler)

    # make rolling sequences (lookback windows) and aligned y
    X_seq, y_seq = [], []
    for i in range(lookback, len(Xs)):
        X_seq.append(Xs[i - lookback:i, :])
        y_seq.append(ys[i, 0])
    if len(X_seq) == 0:
        raise ValueError(f"Not enough rows after FE to form any lookback={lookback} window; "
                         f"provide more history.")
    X_seq = np.array(X_seq, np.float32)         # (N, T, F)
    y_seq = np.array(y_seq,  np.float32).reshape(-1, 1)

    # rebuild + load weights
    model = _rebuild_model(X_seq.shape[-1], best)
    model.load_state_dict(state)
    model.eval()

    # infer in mini-batches
    preds_s = []
    bs = int(best.get("batch", 128))
    with torch.no_grad():
        for i in range(0, len(X_seq), bs):
            xb = torch.from_numpy(X_seq[i:i+bs]).float().to(DEVICE)
            pr = model(xb).cpu().numpy()
            preds_s.append(pr)
    preds_s = np.vstack(preds_s)

    # invert target scaling (+ log if used)
    y_pred = ysc.inverse_transform(preds_s).ravel()
    y_true_scaled = y_seq
    y_true_inv = ysc.inverse_transform(y_true_scaled).ravel()

    if log_tgt:
        y_pred = np.expm1(y_pred)
        y_true_inv = np.expm1(y_true_inv)

    # align timestamps (drop the first 'lookback' FE rows)
    out_idx = dfe.index[lookback:]
    out = pd.DataFrame({
        TIME_COL: dfe.loc[out_idx, TIME_COL].values,
        "y_true": y_true_inv,
        "y_pred": y_pred
    })
    return out


def forecast_multi_steps(model_root: str,
                         df_history: pd.DataFrame,
                         H: int,
                         future_weather_seq: pd.DataFrame | None = None) -> pd.DataFrame:
    """
    Recursive H-step forecast from the end of df_history.
    Inputs:
      - df_history: last N rows with columns [TIMESTAMP, TARGETVAR, U10, V10, U100, V100]
      - future_weather_seq: optional DataFrame with H rows and columns [U10,V10,U100,V100].
        If None, will 'hold' the last known exogenous features each step.
    Output:
      DataFrame with TIMESTAMP (t+1 ... t+H) and y_forecast (in original units).
    """
    best, xsc, ysc, state = load_artifacts(model_root)
    lookback = int(best["lookback"])
    log_tgt  = bool(best["log_target"])

    # we’ll reuse your single-step pipeline each step, updating history with predicted TARGETVAR
    # build model once (we’ll still call FE each step to update lags/rolls)
    # we need input_size; derive it from a one-shot prep using existing history

    # small inner helper to get one-step prediction given (history, optional next exogenous)
    def _one_step(history_df: pd.DataFrame, fw: dict | None):
        # create future placeholder row + FE
        df = history_df.copy().sort_values(TIME_COL).reset_index(drop=True)

        # build future exogenous row
        if fw is None:
            fut = df.iloc[[-1]][[TIME_COL] + BASE_FEATS].copy()
            fut[TIME_COL] = pd.to_datetime(fut[TIME_COL]) + pd.Timedelta(hours=1)
        else:
            fut = pd.DataFrame([{
                TIME_COL: pd.to_datetime(df[TIME_COL].iloc[-1]) + pd.Timedelta(hours=1),
                "U10": fw["U10"], "V10": fw["V10"], "U100": fw["U100"], "V100": fw["V100"]
            }])
        fut[TARGET_COL] = np.nan

        work = pd.concat([df[[TIME_COL, TARGET_COL] + BASE_FEATS], fut], ignore_index=True)

        dfe = add_engineered_features(work)
        dfe_hist = dfe.iloc[:-1].dropna().copy()
        if len(dfe_hist) < lookback:
            # compute minimum rows needed considering max lag/roll (=24 here)
            req = 24 + lookback
            raise ValueError(f"Not enough rows after FE to form lookback={lookback} window. "
                             f"Provide ≥ {req} rows of raw history (have {len(history_df)}).")

        feat_cols = build_feat_list()
        X_hist = dfe_hist[feat_cols].to_numpy(np.float32)
        X_hist_s = xsc.transform(X_hist)
        X_window = X_hist_s[-lookback:, :]
        xb = torch.from_numpy(X_window[None, ...]).float().to(DEVICE)

        return xb, pd.to_datetime(df[TIME_COL].iloc[-1]) + pd.Timedelta(hours=1)

    # initialize model lazily on first step (to know input_size)
    fw0 = None
    if future_weather_seq is not None and len(future_weather_seq) > 0:
        row0 = future_weather_seq.iloc[0]
        fw0 = {"U10": float(row0["U10"]), "V10": float(row0["V10"]),
               "U100": float(row0["U100"]), "V100": float(row0["V100"])}
    xb0, _ = _one_step(df_history, fw0)
    model = _rebuild_model(xb0.shape[-1], best)
    model.load_state_dict(state)
    model.eval()

    preds, times = [], []
    cur_hist = df_history.copy()

    for h in range(H):
        fw = None
        if future_weather_seq is not None:
            row = future_weather_seq.iloc[h]
            fw = {"U10": float(row["U10"]), "V10": float(row["V10"]),
                  "U100": float(row["U100"]), "V100": float(row["V100"])}

        xb, t_next = _one_step(cur_hist, fw)
        with torch.no_grad():
            yhat_s = model(xb).cpu().numpy()
        yhat = ysc.inverse_transform(yhat_s).ravel()[0]
        if log_tgt:
            yhat = np.expm1(yhat)

        preds.append(float(yhat))
        times.append(t_next)

        # append predicted step to history so lags/rolls update
        add_row = cur_hist.iloc[[-1]][[TIME_COL] + BASE_FEATS].copy()
        add_row[TIME_COL] = t_next
        add_row[TARGET_COL] = yhat
        if fw is not None:
            add_row["U10"] = fw["U10"]; add_row["V10"] = fw["V10"]
            add_row["U100"] = fw["U100"]; add_row["V100"] = fw["V100"]
        cur_hist = pd.concat([cur_hist, add_row], ignore_index=True)

    return pd.DataFrame({TIME_COL: times, "y_forecast": preds})



In [6]:
import pandas as pd

# 1) Put your last 24 rows into a DataFrame:
# Must have columns: TIMESTAMP, TARGETVAR, U10, V10, U100, V100
last24 = pd.read_excel("../Predictions/WindPowerForecastingData.xlsx")  # or build manually

# 2) If you already know the next hour’s weather, pass it (optional):
future_weather = {
    "U10": 3.5, "V10": -1.2,
    "U100": 5.1, "V100": -1.8
}
# If you don’t know it, set future_weather=None (the code will hold the last known values)

# 3) Predict the next hour:
next_ts, y_pred = predict_next_from_last24(
    model_root=".",            # folder with best_params.json + biLSTM/*
    last24_df=last24,
    future_weather=None        # or future_weather dict as above
)
print(next_ts, y_pred)


2013-12-01 01:00:00 0.25088977813720703


In [7]:
# # 1) Predict across a whole file
# df_pred = predict_over_file(
#     model_root=".",                             # where ../biLSTM/* is relative to this script
#     data_path="E:/WindPowerForecastingData.xlsx"  # or .csv
#                # or None/0 for CSV
# )
# print(df_pred.head())



In [12]:
# run_future_forecast_uv_hardcoded_opt1.py
# Fill these paths/values and run the script (no argparse).

from pathlib import Path
import pandas as pd
import numpy as np

# ========= HARD-CODE YOUR SETTINGS HERE =========
HISTORY_PATH   = "../Predictions/WindPowerForecastingData.xlsx"  # history with TIMESTAMP, TARGETVAR, U10,V10,U100,V100
FUTURE_UV_PATH = "../Predictions/future_uv.xlsx"                 # exactly U10,V10,U100,V100 (H rows)
H              = 8                                               # forecast horizon (must equal rows in FUTURE_UV_PATH)
MODEL_ROOT     = "."                                             # folder with trained artifacts
OUT_PATH       = "forecast_uv.csv"                               # output (csv or xlsx)
# =================================================

TIME_COL   = "TIMESTAMP"
TARGET_COL = "TARGETVAR"
BASE_FEATS = ["U10","V10","U100","V100"]

def load_table(path: Path) -> pd.DataFrame:
    return pd.read_excel(path) if path.suffix.lower() in (".xlsx", ".xls") else pd.read_csv(path)

def sanitize_history(df: pd.DataFrame, time_col: str = TIME_COL) -> pd.DataFrame:
    df = df.copy()
    df[time_col] = pd.to_datetime(df[time_col], errors="coerce")
    bad = df[time_col].isna()
    if bad.any():
        ex = df.loc[bad, time_col].head(3).tolist()
        raise ValueError(f"{bad.sum()} {time_col} values could not be parsed. Examples: {ex}")
    for c in [*BASE_FEATS, TARGET_COL]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    return df.sort_values(time_col).reset_index(drop=True)

def trim_trailing_target_nans(df: pd.DataFrame, target_col: str = TARGET_COL) -> pd.DataFrame:
    df = df.copy()
    last_valid = df[target_col].last_valid_index()
    if last_valid is None:
        raise ValueError("No valid TARGETVAR in history.")
    return df.loc[:last_valid].reset_index(drop=True)

# 0) Load BOTH: raw (untrimmed) and trimmed (for the model)
hist_raw = load_table(Path(HISTORY_PATH))
hist_raw = sanitize_history(hist_raw, time_col=TIME_COL)   # DO NOT TRIM here

hist = trim_trailing_target_nans(hist_raw, target_col=TARGET_COL)

# 1) Load future U/V and validate
fw = load_table(Path(FUTURE_UV_PATH))
missing = [c for c in BASE_FEATS if c not in fw.columns]
if missing:
    raise KeyError(f"--future_uv missing required columns: {missing}. Found: {list(fw.columns)}")

fw = fw[BASE_FEATS].copy()
for c in BASE_FEATS:
    fw[c] = pd.to_numeric(fw[c], errors="coerce")
if fw.isna().any().any():
    bad_cols = fw.columns[fw.isna().any()].tolist()
    raise ValueError(f"--future_uv contains NaNs in columns: {bad_cols}")
if len(fw) != H:
    raise ValueError(f"--future_uv has {len(fw)} rows but H={H}. They must match exactly.")


df_fore = forecast_multi_steps(
    model_root=MODEL_ROOT,
    df_history=hist,        # trimmed for valid lags
    H=H,
    future_weather_seq=fw
)

# 3) Anchor timestamps to the FILE'S true last time (OPTION 1)
#    This is the max timestamp in the sanitized, untrimmed file.
anchor_ts = pd.to_datetime(hist_raw[TIME_COL]).max()
ts_future = pd.date_range(anchor_ts + pd.Timedelta(hours=1), periods=H, freq="H")

# Insert/replace TIMESTAMP in the output
df_fore = df_fore.copy()
if TIME_COL in df_fore.columns:
    df_fore.drop(columns=[TIME_COL], inplace=True, errors="ignore")
df_fore.insert(0, TIME_COL, ts_future)

# 4) Save
out_path = Path(OUT_PATH)
if out_path.suffix.lower() in (".xlsx", ".xls"):
    df_fore.to_excel(out_path, index=False)
else:
    df_fore.to_csv(out_path, index=False)

print(f"Saved: {out_path.resolve()}")
print(df_fore.head(30))


Saved: /home/sulith/LSTM/LSTM_Wind_assignment/Predictions/forecast_uv.csv
            TIMESTAMP  y_forecast
0 2013-12-01 01:00:00    0.183762
1 2013-12-01 02:00:00    0.219495
2 2013-12-01 03:00:00    0.336016
3 2013-12-01 04:00:00    0.405033
4 2013-12-01 05:00:00    0.495598
5 2013-12-01 06:00:00    0.497326
6 2013-12-01 07:00:00    0.489461
7 2013-12-01 08:00:00    0.450025


/tmp/ipykernel_1003/467537932.py:74: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  ts_future = pd.date_range(anchor_ts + pd.Timedelta(hours=1), periods=H, freq="H")


In [10]:
df_pred = predict_over_file(model_root=".", data_path=r"../Predictions/WindPowerForecastingData.xlsx")

import numpy as np

# Align arrays (if needed)
min_len = min(len(df_pred["y_true"]), len(df_pred["y_pred"]))
y_true_aligned = np.asarray(df_pred["y_true"]).reshape(-1)[:min_len]
y_pred_aligned = np.asarray(df_pred["y_pred"]).reshape(-1)[:min_len]

# Save to CSV
forecast_df = pd.DataFrame({
    "TIMESTAMP": df_pred["TIMESTAMP"].values[:min_len],
    "Actual": y_true_aligned,
    "Predicted": y_pred_aligned
})
forecast_df.to_csv("forecast_results_bilstm.csv", index=False)




/tmp/ipykernel_1003/2589631756.py:104: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[TIME_COL] = pd.to_datetime(df[TIME_COL], errors="coerce", infer_datetime_format=True)


In [11]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Extract actual and predicted values
y_true = df_pred["y_true"].values
y_pred = df_pred["y_pred"].values

# Calculate error metrics
mae = mean_absolute_error(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_true, y_pred)

# Print the metrics
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R² Score: {r2:.4f}")


Mean Absolute Error (MAE): 0.0766
Mean Squared Error (MSE): 0.0132
Root Mean Squared Error (RMSE): 0.1148
R² Score: 0.8426
